In [1]:
from helper_classes import Data
import util as ut

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten,Dropout
from tensorflow.keras.regularizers import l2


import warnings
import logging
from scipy import stats
import numpy as np
import random
import torch
from sklearn.preprocessing import MultiLabelBinarizer
random_state = 1
np.random.seed(random_state)
random.seed(random_state)
warnings.filterwarnings('ignore')

In [2]:
dataset='WN18RR'
kg_root = 'KGs/'+dataset+'/'
dataset=Data(kg_root)

In [3]:
y=[]
x=[]

entitiy_idx=dict()

sub_obj_pairs= dataset.get_entity_pairs_with_predicates(dataset.train_data)
for s_o_pair, predicates in sub_obj_pairs.items():
    s,o=s_o_pair
    entitiy_idx.setdefault(s, len(entitiy_idx))
    entitiy_idx.setdefault(o, len(entitiy_idx))
    x.append([entitiy_idx[s],entitiy_idx[o]])
    y.append(list(predicates))
x=np.array(x)

In [4]:
print(x[0]) # input
print(y[0]) # output

[0 1]
['_hypernym']


In [5]:
x=np.array(x)
binarizer=MultiLabelBinarizer()
y=binarizer.fit_transform(y)
print(x[0]) # input
print(y[0]) # output

[0 1]
[0 0 0 1 0 0 0 0 0 0 0]


In [6]:
binarizer.classes_

array(['_also_see', '_derivationally_related_form', '_has_part',
       '_hypernym', '_instance_hypernym', '_member_meronym',
       '_member_of_domain_region', '_member_of_domain_usage',
       '_similar_to', '_synset_domain_topic_of', '_verb_group'],
      dtype=object)

In [7]:
embedding_dim=100
model = Sequential()
model.add(Embedding(len(entitiy_idx), embedding_dim, input_length=2,activity_regularizer=l2(0.1)))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(embedding_dim*3, activation='relu',activity_regularizer=l2(0.1)))
model.add(Dropout(.5))
model.add(Dense(y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',  # adam
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 100)            4055900   
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               60300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 11)                3311      
Total params: 4,119,511
Trainable params: 4,119,511
Non-trainable params: 0
______________________________________________

In [8]:
history = model.fit(x, y, batch_size=512, epochs= 100, use_multiprocessing=True,verbose=1,shuffle=True)

Train on 86726 samples
Epoch 1/100
86726/86726 [==============================] - 6s 75us/sample - loss: 1.8792 - accuracy: 0.4132
Epoch 2/100
86726/86726 [==============================] - 6s 70us/sample - loss: 1.1737 - accuracy: 0.6340
Epoch 3/100
86726/86726 [==============================] - 7s 77us/sample - loss: 0.7872 - accuracy: 0.8217
Epoch 4/100
86726/86726 [==============================] - 6s 69us/sample - loss: 0.5990 - accuracy: 0.8705
Epoch 5/100
86726/86726 [==============================] - 6s 64us/sample - loss: 0.4938 - accuracy: 0.8990
Epoch 6/100
86726/86726 [==============================] - 6s 64us/sample - loss: 0.4250 - accuracy: 0.9182
Epoch 7/100
86726/86726 [==============================] - 6s 64us/sample - loss: 0.3760 - accuracy: 0.9321
Epoch 8/100
86726/86726 [==============================] - 6s 65us/sample - loss: 0.3383 - accuracy: 0.9412
Epoch 9/100
86726/86726 [==============================] - 6s 67us/sample - loss: 0.3104 - accuracy: 0.9466
Epoch

In [9]:
def evaluation(model,binarizer,dataset,triples):
    x_=[]
    y_=[]
    
    hits = []
    ranks=[]
    for i in range(10):
        hits.append([])

    rank_per_relation=dict()
        
    for i in triples:
        s,p,o=i
        x_.append((entitiy_idx[s],entitiy_idx[o]))
        y_.append(p)
    
    
    tensor_pred=torch.from_numpy(model.predict(np.array(x_)))
    
    _, ranked_predictions =tensor_pred.topk(k=len(binarizer.classes_))
    
    ranked_predictions=ranked_predictions.numpy()
    
    
    assert len(ranked_predictions)==len(y_)
    
    classes_=binarizer.classes_.tolist()
        
    for i in range(len(y_)):
        true_relation=y_[i]
        ith_class=classes_.index(true_relation)
        
        rank = np.where(ranked_predictions[i]==ith_class)[0]
        

        rank_per_relation.setdefault(true_relation, []).append(rank+1)
        
        ranks.append(rank+1)
        
        for hits_level in range(10):
            if rank <= hits_level:
                hits[hits_level].append(1.0)
    

    hits=np.array(hits)
    ranks=np.array(ranks)
    print('########## Relation Prediction Results ##########')

    print('Mean Hits @5: {0}'.format(sum(hits[4]) / (float(len(y_)))))
    print('Mean Hits @3: {0}'.format(sum(hits[2]) / (float(len(y_)))))
    print('Mean @1: {0}'.format(sum(hits[0]) / (float(len(y_)))))
    print('Mean rank: {0}'.format(np.mean(ranks)))
    print('Mean reciprocal rank: {0}'.format(np.mean(1. / ranks)))
    
    print('########## Relation Prediction Analysis ##########')
    
    for pred, ranks in rank_per_relation.items():
        ranks=np.array(ranks)
        
        average_hit_at_1=np.sum(ranks==1)/len(ranks)
        average_hit_at_3=np.sum(ranks<=3)/len(ranks)
        average_hit_at_5=np.sum(ranks<=5)/len(ranks)
        
        print('{0}:\t Hits@1:\t{1:.3f}'.format(pred,average_hit_at_1))
        print('{0}:\t Hits@3:\t{1:.3f}'.format(pred,average_hit_at_3))
        print('{0}:\t Hits@5:\t{1:.3f}'.format(pred,average_hit_at_5))
        print('{0}:\t MRR:\t{1:.3f}\t number of occurrence {2}'.format(pred,np.mean(1. / ranks),len(ranks)))
        print('################################')
    

In [10]:
evaluation(model,binarizer,dataset,dataset.train_data)

########## Relation Prediction Results ##########
Mean Hits @5: 1.0
Mean Hits @3: 1.0
Mean @1: 0.9984913917199286
Mean rank: 1.0015086082800715
Mean reciprocal rank: 0.9992456958599643
########## Relation Prediction Analysis ##########
_hypernym:	 Hits@1:	0.999
_hypernym:	 Hits@3:	1.000
_hypernym:	 Hits@5:	1.000
_hypernym:	 MRR:	1.000	 number of occurrence 34796
################################
_derivationally_related_form:	 Hits@1:	0.999
_derivationally_related_form:	 Hits@3:	1.000
_derivationally_related_form:	 Hits@5:	1.000
_derivationally_related_form:	 MRR:	0.999	 number of occurrence 29715
################################
_instance_hypernym:	 Hits@1:	1.000
_instance_hypernym:	 Hits@3:	1.000
_instance_hypernym:	 Hits@5:	1.000
_instance_hypernym:	 MRR:	1.000	 number of occurrence 2921
################################
_also_see:	 Hits@1:	0.995
_also_see:	 Hits@3:	1.000
_also_see:	 Hits@5:	1.000
_also_see:	 MRR:	0.998	 number of occurrence 1299
################################
_membe

In [11]:
evaluation(model,binarizer,dataset,dataset.valid_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9946883852691218
Mean Hits @3: 0.9812322946175638
Mean @1: 0.8753541076487252
Mean rank: 1.1993626062322946
Mean reciprocal rank: 0.9295466297945053
########## Relation Prediction Analysis ##########
_hypernym:	 Hits@1:	0.828
_hypernym:	 Hits@3:	0.979
_hypernym:	 Hits@5:	0.996
_hypernym:	 MRR:	0.906	 number of occurrence 1010
################################
_derivationally_related_form:	 Hits@1:	0.950
_derivationally_related_form:	 Hits@3:	0.997
_derivationally_related_form:	 Hits@5:	0.998
_derivationally_related_form:	 MRR:	0.973	 number of occurrence 1076
################################
_synset_domain_topic_of:	 Hits@1:	0.990
_synset_domain_topic_of:	 Hits@3:	1.000
_synset_domain_topic_of:	 Hits@5:	1.000
_synset_domain_topic_of:	 MRR:	0.995	 number of occurrence 102
################################
_member_meronym:	 Hits@1:	0.803
_member_meronym:	 Hits@3:	0.962
_member_meronym:	 Hits@5:	0.989
_member_meronym:	 MRR:	0

In [12]:
evaluation(model,binarizer,dataset,dataset.test_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9952120383036935
Mean Hits @3: 0.9829001367989056
Mean @1: 0.874829001367989
Mean rank: 1.1932284541723666
Mean reciprocal rank: 0.9295151781642889
########## Relation Prediction Analysis ##########
_member_of_domain_usage:	 Hits@1:	0.955
_member_of_domain_usage:	 Hits@3:	1.000
_member_of_domain_usage:	 Hits@5:	1.000
_member_of_domain_usage:	 MRR:	0.977	 number of occurrence 22
################################
_verb_group:	 Hits@1:	0.744
_verb_group:	 Hits@3:	1.000
_verb_group:	 Hits@5:	1.000
_verb_group:	 MRR:	0.868	 number of occurrence 39
################################
_member_of_domain_region:	 Hits@1:	0.885
_member_of_domain_region:	 Hits@3:	0.962
_member_of_domain_region:	 Hits@5:	0.962
_member_of_domain_region:	 MRR:	0.927	 number of occurrence 26
################################
_member_meronym:	 Hits@1:	0.785
_member_meronym:	 Hits@3:	0.947
_member_meronym:	 Hits@5:	0.992
_member_meronym:	 MRR:	0.872	 number o